In [20]:
# %%
# 📘 Step 1: Import libraries
import pandas as pd
import re

# %%
# 📘 Step 2: File paths
excel_file = r"C:\Users\SUBARNA MONDAL\Desktop\New folder (2)\Domain Leads151025(OLD_USA).xlsx"
template_csv = r"C:\Users\SUBARNA MONDAL\Desktop\New folder (2)\campaign-template.csv"

# %%
# 📘 Step 3: Load Excel and CSV
df_excel = pd.read_excel(excel_file, dtype=str)
df_template = pd.read_csv(template_csv, dtype=str)

print("✅ Excel columns loaded:", df_excel.columns.tolist())

# %%
# 📘 Step 4: Clean and normalize phone numbers (must start with '1')
def clean_us_phone(phone):
    if pd.isna(phone):
        return ""
    phone = str(phone)
    digits = re.sub(r'\D', '', phone)

    # 11-digit (starting with 1) stays same
    if digits.startswith("1") and len(digits) == 11:
        return digits
    # 10-digit gets "1" prefix
    elif len(digits) == 10:
        return f"1{digits}"
    # fallback for others
    return digits

# %%
# 📘 Step 5: Main extraction and fill function
def extract_and_fill(start, end):
    # Select row range
    selected = df_excel.iloc[start-1:end].copy()

    # Ensure required columns exist
    for col in ['domain_name', 'registrant_name', 'registrant_phone']:
        if col not in selected.columns:
            raise KeyError(f"Missing column: {col}")

    # Clean phone numbers
    selected['registrant_phone'] = selected['registrant_phone'].apply(clean_us_phone)

    # Build the new DataFrame matching the template exactly
    new_data = pd.DataFrame({
        'number': selected['registrant_phone'],
        'name': selected['registrant_name'],
        'another_var': selected['domain_name']  # ✅ fixed key name
    })

    # Drop dummy or test rows from the template
    df_clean_template = df_template[
        ~df_template['name'].astype(str).str.lower().isin(['john doe', 'jane smith', 'bob johnson'])
    ]

    # Combine clean template and new data
    combined = pd.concat([df_clean_template, new_data], ignore_index=True)

    # Make sure only the right columns exist
    combined = combined[['number', 'name', 'another_var']]

    return combined

# %%
# 📘 Step 6: Get range input
user_input = input("Enter range (e.g., 500-510): ").strip()
start, end = map(int, user_input.split('-'))

# %%
# 📘 Step 7: Extract and save clean CSV
output_df = extract_and_fill(start, end)

output_filename = fr"C:\Users\SUBARNA MONDAL\Desktop\New folder (2)\campaign-output-{start}-{end}.csv"
output_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"✅ Done! Extracted rows {start}-{end} and saved to:")
print(output_filename)


✅ Excel columns loaded: ['domain_name', 'create_date', 'expiry_date', 'domain_registrar_name', 'registrant_name', 'registrant_company', 'registrant_address', 'registrant_city', 'registrant_state', 'registrant_zip', 'registrant_country', 'registrant_email', 'registrant_phone']
✅ Done! Extracted rows 588-598 and saved to:
C:\Users\SUBARNA MONDAL\Desktop\New folder (2)\campaign-output-588-598.csv
